In [1]:
from __future__ import print_function

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [3]:
from ilqr.costs.finite_diff import FiniteDiffCost
from ilqr.mujoco_dynamics import MujocoDynamics
from ilqr.mujoco_controller import iLQR, RecedingHorizonController
from ilqr.dynamics import constrain
from ilqr.utils.visualization import make_video_fn

from scipy.optimize import approx_fprime

import mujoco_py
from mujoco_py import MjViewer
import os

%load_ext autoreload
%autoreload 2

In [4]:
def on_iteration(iteration_count, xs, us, J_opt, accepted, converged):
    J_hist.append(J_opt)
    info = "converged" if converged else ("accepted" if accepted else "failed")
    final_state = xs[-1]
    print("iteration", iteration_count, info, J_opt, final_state)

In [5]:
xml_path = os.path.join('..', 'ilqr', 'xmls', 'reacher.xml')

#STATE: [theta1, theta2, goal_x, goal_y, theta1_dot, theta2_dot, goal_x_dot, goal_y_dot]

dynamics = MujocoDynamics(xml_path, frame_skip = 2, use_multiprocessing = True)
print('dt:', dynamics.dt)

Finished loading process4070 Finished loading process
 4071Finished loading process 
4072Finished loading process
 Finished loading process4073 
4074
Finished loading process 4075Finished loading process 
Finished loading process4076 
4077Finished loading process
 4078
Finished loading process 4079Finished loading process 
4080dt: 0.02

Finished loading process 4081Finished loading process
 4082
Finished loading process Finished loading process4083
 4084
Finished loading process 4085


In [6]:
def l(x, u, i):
    jointposx = 0.1 * np.cos(x[0])
    jointposy = 0.1 * np.sin(x[0])
    fingerposx = jointposx + 0.1 * np.cos(x[0] + x[1])
    fingerposy = jointposy + 0.1 * np.sin(x[0] + x[1])
    
    dist = np.sqrt((fingerposx - x[2]) ** 2 + (fingerposy - x[3]) ** 2)
    return 100 * (dist) + x[4] ** 2 + x[5] ** 2 + u[0] ** 2 + u[1] ** 2

cost2 = FiniteDiffCost(l, lambda x, i: l(x, [0.0, 0.0], i), 8, 2, use_multiprocessing = True)

Finished loading process Finished loading process4086 
Finished loading process4087
 4088Finished loading process
 4089Finished loading process
 4090Finished loading process
 4091
Finished loading process 4092Finished loading process 
Finished loading process4093 
4094Finished loading process
 4095Finished loading process
 4096Finished loading process
 4097Finished loading process
 4098Finished loading process
 4099
Finished loading process 4100Finished loading process
 4101


In [7]:
N = 100
x0 = np.array([0.0, 0.0, np.random.uniform(-0.14, 0.14), np.random.uniform(-0.14, 0.14), 0.0, 0.0, 0.0, 0.0])
#us_init = np.random.uniform(-1, 1, (N, dynamics.action_size))
us_init = np.zeros((N, dynamics.action_size))
ilqr = iLQR(dynamics, cost2, N, multiprocessing = True)
mpc = RecedingHorizonController(x0, ilqr)

Finished loading process 4102Finished loading process
 Finished loading process4103 4104
Finished loading process
 Finished loading process 4105
4106Finished loading process
 4107
Finished loading process 4108Finished loading process
 4109
Finished loading process 4110
Finished loading process 4111Finished loading process
 4112Finished loading process
 4113Finished loading process
 4114
Finished loading process 4115
Finished loading process 4116Finished loading process
 4117


In [8]:
t0 = time.time()
J_hist = []
mpc_trajectory, controls = mpc.control(us_init, 50, step_size = 1, initial_n_iterations = 500, subsequent_n_iterations = 100, on_iteration = on_iteration)
us = []

print('time', time.time() - t0)

iteration 0 accepted 1097.9603251842047 [ 1.39196051e+00 -4.07465126e-01  8.10334631e-02  7.75694739e-02
 -4.78799268e-04 -3.16312993e-03  0.00000000e+00  0.00000000e+00]
iteration 1 accepted 1009.0686599104597 [ 6.05796153e-01 -2.29224815e-01  8.10334631e-02  7.75694739e-02
  4.57396279e-04 -8.09462447e-03  0.00000000e+00  0.00000000e+00]
iteration 2 accepted 984.5790334794845 [ 0.9291621  -0.27792703  0.08103346  0.07756947  0.00411237 -0.00579839
  0.          0.        ]
iteration 3 accepted 983.8655250936719 [ 0.88565244 -0.28466003  0.08103346  0.07756947  0.00292229 -0.00649516
  0.          0.        ]
iteration 4 failed 983.8655250936716 [ 0.88565244 -0.28466003  0.08103346  0.07756947  0.00292229 -0.00649516
  0.          0.        ]
iteration 5 failed 983.8655250936716 [ 0.88565244 -0.28466003  0.08103346  0.07756947  0.00292229 -0.00649516
  0.          0.        ]
iteration 6 failed 983.8655250936716 [ 0.88565244 -0.28466003  0.08103346  0.07756947  0.00292229 -0.00649516


iteration 48 accepted 562.4668582468585 [-0.22749771  1.8298633   0.08103346  0.07756947  0.01082372 -0.02223446
  0.          0.        ]
iteration 49 accepted 525.6369433490959 [-0.2160546   1.96325113  0.08103346  0.07756947  0.0104334  -0.02113597
  0.          0.        ]
iteration 50 accepted 520.4448164383808 [-0.20305075  1.9227987   0.08103346  0.07756947  0.01313169 -0.02014936
  0.          0.        ]
iteration 51 accepted 520.3434521728279 [-0.21982964  1.98176153  0.08103346  0.07756947  0.00763573 -0.01442195
  0.          0.        ]
iteration 52 accepted 518.6535349091935 [-0.20704897  1.92376596  0.08103346  0.07756947  0.00924097 -0.01637823
  0.          0.        ]
iteration 53 accepted 514.216972903312 [-0.21161728  1.95233831  0.08103346  0.07756947  0.00802424 -0.01412103
  0.          0.        ]
iteration 54 accepted 513.9618059879049 [-0.21209807  1.94721831  0.08103346  0.07756947  0.0060994  -0.01480604
  0.          0.        ]
iteration 55 accepted 513.72

iteration 22 accepted 490.3921310437732 [-0.21139903  1.95082348  0.08103346  0.07756947  0.02410558  0.03414816
  0.          0.        ]
iteration 23 converged 490.391941573014 [-0.21139947  1.95082337  0.08103346  0.07756947  0.02409812  0.0341414
  0.          0.        ]
iteration 0 accepted 465.78564238844365 [-0.21194017  1.95102128  0.08103346  0.07756947  0.02274798  0.05625962
  0.          0.        ]
iteration 1 accepted 465.7830138672049 [-0.21180237  1.95067691  0.08103346  0.07756947  0.01322249  0.02469961
  0.          0.        ]
iteration 2 failed 465.78301386720494 [-0.21180237  1.95067691  0.08103346  0.07756947  0.01322249  0.02469961
  0.          0.        ]
iteration 3 failed 465.78301386720494 [-0.21180237  1.95067691  0.08103346  0.07756947  0.01322249  0.02469961
  0.          0.        ]
iteration 4 failed 465.78301386720494 [-0.21180237  1.95067691  0.08103346  0.07756947  0.01322249  0.02469961
  0.          0.        ]
iteration 5 failed 465.783013867204

iteration 10 accepted 327.3958878983543 [-0.21190153  1.95071731  0.08103346  0.07756947 -0.03279179  0.00788384
  0.          0.        ]
iteration 11 failed 327.39588789835443 [-0.21190153  1.95071731  0.08103346  0.07756947 -0.03279179  0.00788384
  0.          0.        ]
iteration 12 failed 327.39588789835443 [-0.21190153  1.95071731  0.08103346  0.07756947 -0.03279179  0.00788384
  0.          0.        ]
iteration 13 failed 327.39588789835443 [-0.21190153  1.95071731  0.08103346  0.07756947 -0.03279179  0.00788384
  0.          0.        ]
iteration 14 accepted 327.39469226103836 [-0.21190152  1.95071747  0.08103346  0.07756947 -0.03278941  0.00788325
  0.          0.        ]
iteration 15 accepted 327.3942328947944 [-0.21190181  1.9507185   0.08103346  0.07756947 -0.03276544  0.00787938
  0.          0.        ]
iteration 16 failed 327.3942328947944 [-0.21190181  1.9507185   0.08103346  0.07756947 -0.03276544  0.00787938
  0.          0.        ]
iteration 17 failed 327.3942328

iteration 5 converged 221.2084504793725 [-2.11885553e-01  1.95110613e+00  8.10334631e-02  7.75694739e-02
 -4.99141818e-04  1.96245453e-02  0.00000000e+00  0.00000000e+00]
iteration 0 accepted 206.14438872028708 [-2.11853851e-01  1.95098969e+00  8.10334631e-02  7.75694739e-02
  1.14532188e-03  2.56295904e-02  0.00000000e+00  0.00000000e+00]
iteration 1 converged 206.14426365563872 [-2.11847338e-01  1.95079549e+00  8.10334631e-02  7.75694739e-02
  1.71472175e-03  1.37548618e-02  0.00000000e+00  0.00000000e+00]
iteration 0 accepted 191.772852830714 [-0.21176262  1.95117169  0.08103346  0.07756947  0.00671677  0.02551227
  0.          0.        ]
iteration 1 accepted 191.76641070423818 [-2.11813054e-01  1.95067115e+00  8.10334631e-02  7.75694739e-02
  9.68900241e-04  1.17269492e-02  0.00000000e+00  0.00000000e+00]
iteration 2 failed 191.76641070423815 [-2.11813054e-01  1.95067115e+00  8.10334631e-02  7.75694739e-02
  9.68900241e-04  1.17269492e-02  0.00000000e+00  0.00000000e+00]
iteration

...

iteration 47 accepted 4.957458485708059 [-0.2112909   1.94977643  0.08103346  0.07756947  0.04909869 -0.09958894
  0.          0.        ]
iteration 48 accepted 4.95728887955616 [-0.21129095  1.94977651  0.08103346  0.07756947  0.04909793 -0.09958697
  0.          0.        ]
iteration 49 accepted 4.957144435365066 [-0.21129112  1.94977684  0.08103346  0.07756947  0.04909245 -0.09957433
  0.          0.        ]
iteration 50 accepted 4.955753846601043 [-0.21131465  1.94982096  0.08103346  0.07756947  0.0474472  -0.09606454
  0.          0.        ]
iteration 51 accepted 4.9469348714774055 [-0.21167769  1.95049538  0.08103346  0.07756947  0.02068906 -0.04152185
  0.          0.        ]
iteration 52 failed 4.946934871477407 [-0.21167769  1.95049538  0.08103346  0.07756947  0.02068906 -0.04152185
  0.          0.        ]
iteration 53 failed 4.946934871477407 [-0.21167769  1.95049538  0.08103346  0.07756947  0.02068906 -0.04152185
  0.          0.        ]
iteration 54 failed 4.946934871

iteration 8 accepted 2.8159556705543674 [-0.21169983  1.95022148  0.08103346  0.07756947  0.02863009 -0.07261824
  0.          0.        ]
iteration 9 failed 2.815955670554368 [-0.21169983  1.95022148  0.08103346  0.07756947  0.02863009 -0.07261824
  0.          0.        ]
iteration 10 failed 2.815955670554368 [-0.21169983  1.95022148  0.08103346  0.07756947  0.02863009 -0.07261824
  0.          0.        ]
iteration 11 failed 2.815955670554368 [-0.21169983  1.95022148  0.08103346  0.07756947  0.02863009 -0.07261824
  0.          0.        ]
iteration 12 accepted 2.8158470058738145 [-0.2116998   1.9502216   0.08103346  0.07756947  0.02862939 -0.07261504
  0.          0.        ]
iteration 13 accepted 2.8158421005196734 [-0.21169979  1.95022178  0.08103346  0.07756947  0.02862814 -0.07260895
  0.          0.        ]
iteration 14 accepted 2.8157224967941707 [-0.21170002  1.9502236   0.08103346  0.07756947  0.02861285 -0.07251547
  0.          0.        ]
iteration 15 failed 2.815722496

iteration 69 accepted 2.778890288788364 [-0.21177235  1.95062183  0.08103346  0.07756947  0.00483482 -0.02257262
  0.          0.        ]
iteration 70 accepted 2.7788587333406047 [-0.21177235  1.95062184  0.08103346  0.07756947  0.00483461 -0.02257027
  0.          0.        ]
iteration 71 accepted 2.7788415729488065 [-0.21177234  1.95062185  0.08103346  0.07756947  0.0048329  -0.02256181
  0.          0.        ]
iteration 72 failed 2.7788415729488043 [-0.21177234  1.95062185  0.08103346  0.07756947  0.0048329  -0.02256181
  0.          0.        ]
iteration 73 failed 2.7788415729488043 [-0.21177234  1.95062185  0.08103346  0.07756947  0.0048329  -0.02256181
  0.          0.        ]
iteration 74 failed 2.7788415729488043 [-0.21177234  1.95062185  0.08103346  0.07756947  0.0048329  -0.02256181
  0.          0.        ]
iteration 75 failed 2.7788415729488043 [-0.21177234  1.95062185  0.08103346  0.07756947  0.0048329  -0.02256181
  0.          0.        ]
iteration 76 accepted 2.77883

iteration 32 accepted 1.996707163849341 [-0.21165153  1.94995109  0.08103346  0.07756947  0.00787987 -0.0499801
  0.          0.        ]
iteration 33 accepted 1.9966765263833866 [-0.21165174  1.949953    0.08103346  0.07756947  0.0078829  -0.04993427
  0.          0.        ]
iteration 34 accepted 1.9966392859883126 [-0.21165357  1.94996747  0.08103346  0.07756947  0.00790551 -0.04919312
  0.          0.        ]
iteration 35 accepted 1.9966358595780578 [-0.21165367  1.94996824  0.08103346  0.07756947  0.00790615 -0.0491504
  0.          0.        ]
iteration 36 accepted 1.9965573737695217 [-0.21165418  1.94997216  0.08103346  0.07756947  0.00790935 -0.04893301
  0.          0.        ]
iteration 37 accepted 1.996516887267197 [-0.2116547   1.94997602  0.08103346  0.07756947  0.00791448 -0.04871569
  0.          0.        ]
iteration 38 accepted 1.9965097987842522 [-0.2116548   1.9499768   0.08103346  0.07756947  0.00791502 -0.04867287
  0.          0.        ]
iteration 39 accepted 1.

iteration 51 accepted 1.3764744531154367 [-0.21161934  1.95008342  0.08103346  0.07756947  0.01131741 -0.05725841
  0.          0.        ]
iteration 52 accepted 1.3764460753802603 [-0.21161911  1.95008376  0.08103346  0.07756947  0.01131746 -0.05725678
  0.          0.        ]
iteration 53 accepted 1.3760859585096197 [-0.21161902  1.95008409  0.08103346  0.07756947  0.01131754 -0.05725303
  0.          0.        ]
iteration 54 failed 1.3760859585096197 [-0.21161902  1.95008409  0.08103346  0.07756947  0.01131754 -0.05725303
  0.          0.        ]
iteration 55 failed 1.3760859585096197 [-0.21161902  1.95008409  0.08103346  0.07756947  0.01131754 -0.05725303
  0.          0.        ]
iteration 56 accepted 1.3757195480848814 [-0.21161902  1.95008426  0.08103346  0.07756947  0.01131756 -0.05725124
  0.          0.        ]
iteration 57 accepted 1.3755939446803696 [-0.21161902  1.95008433  0.08103346  0.07756947  0.01131756 -0.05725054
  0.          0.        ]
iteration 58 accepted 1.

In [9]:
from mujoco_py import MjViewer
viewer = MjViewer(dynamics.sim)
dynamics.set_state(x0)
print(dynamics.get_state())
video = []
for i in range(100):
    viewer.render()
for c in controls:
    dynamics.step(c)
    viewer.render()

Creating window glfw
[0.         0.         0.08103346 0.07756947 0.         0.
 0.         0.        ]
